# Prediction of Protein Interaction Network

Machine learning classification task for predicting whether a protein pair interacts or not

Protein pairs are generated from a test pathogen against human proteins contained in training set

In [1]:
import os
import joblib
import itertools
from functools import partial
from multiprocessing import Pool
from tqdm import tqdm
from time import time

import pandas as pd
import numpy as np
from scipy import sparse

from Bio import SearchIO

from features import domain_features

In [2]:
# Set up directories
parent_dir = os.path.dirname(os.getcwd())

dir_in = os.path.join(parent_dir, 'data')
dir_out = os.path.join(parent_dir, 'data', 'results')

## Obtain candidate protein domains

Protein sequence source: https://www.uniprot.org

Pathogen: *Streptococcus pneumoniae* strain D39 (**STRP2**; Taxonomy ID 373153)
>Keyword filters
- `Organism [OS]: Streptococcus pneumoniae serotype 2 (strain D39 / NCTC 7466) [373153]`
- `Sequence length: From 50`
- `Gene Ontology [GO]: extracellular region [5576] (20), pathogenesis [9405] (3)`
- `All: choline (9), sialidase (2), amidase activity (4), virulence (4), surface protein (2), adhesion (3)`
- Total: 42 proteins

Host: *Homo sapiens* (Taxonomy ID 9606)
>Keyword filters
- `Organism [OS]: Homo sapiens (Human) [9606]`
- `Proteome ID: up000005640`
- `Sequence length: From 50`
- `All: extracellular`
- `Gene Ontology [GO]: immunoglobulin receptor activity [19763] (6), regulation of complement activation [30449] (120), phagocytosis [6909] (204), toll-like receptor signalling pathway [2224] (54), plasminogen activation [31639] (15), defense response to Gram-positive bacterium [50830] (87)`
- Total: 395 proteins


Extraction of Pfam domains: `hmmscan`
- `hmmscan --tblout STRP2_pfam_hits --acc --noali -E 0.00001 --domE 0.00001 --cpu 7 ~/hmmer-3.2.1/pfam/Pfam-A.hmm STRP2_sequences.fasta`

In [3]:
# Parse hmmscan result of STRP2
pfam_set = joblib.load('pfam.pkl')[1] # Pfam domains available in the training dataset
prot_lists = {}
pfam_dict = {} # generate a new dict for the candidate proteins

for organism in ['STRP2', 'HUMAN']:
    f_in = os.path.join(dir_in, '%s_pfam_hits' % organism)
    proteins = [] # store Uniprot accessions of the current organism
    
    for query in SearchIO.parse(f_in, 'hmmer3-tab'):
        uniprot_id = query.id.split('|')[2] # use protein name to ease network analysis
        domains = []

        # Read each domain hits in query
        for hit in query.hits:
            pfam_acc = hit.accession.split('.')[0] # Pfam accession of domain

            # Select only domains that exists in the training set
            if pfam_acc in pfam_set: 
                domains.append(pfam_acc)

        # Add the pathogen protein to an existing domains dict
        pfam_dict[uniprot_id] = domains 
        proteins.append(uniprot_id)
    
    # Store protein list of the current organism in a dict
    prot_lists[organism] = proteins
    
    # Print statistics
    print('Selected %i %s proteins for HP-PPI prediction' % (len(proteins), organism))

Selected 42 STRP2 proteins for HP-PPI prediction
Selected 388 HUMAN proteins for HP-PPI prediction


## Generate candidate protein pairs and extract domain features

In [4]:
# Generate protein pairs
pairs = [pair for pair in itertools.product(prot_lists['STRP2'], prot_lists['HUMAN'])]
print('Generated %i protein pairs for prediction' % len(pairs))

Generated 16296 protein pairs for prediction


In [5]:
# Set up feature extraction function
feature_function = partial(domain_features,
                           domain_dict=pfam_dict,
                           domain_set=pfam_set)

# Get features of each protein as dict to speed up
# extraction from pairs
all_prots = prot_lists['STRP2'] + prot_lists['HUMAN']
feat_dict = {prot: feature_function(prot) for prot in tqdm(all_prots)}

100%|██████████| 430/430 [00:00<00:00, 723.21it/s]


## HP-PPI Prediction

In [6]:
# Load pre-trained classifier
clf = joblib.load('best_model.pkl')
clf.get_params()

{'Cs': array([1.00000000e-02, 2.51188643e-02, 6.30957344e-02, 1.58489319e-01,
        3.98107171e-01, 1.00000000e+00, 2.51188643e+00, 6.30957344e+00,
        1.58489319e+01, 3.98107171e+01, 1.00000000e+02]),
 'class_weight': None,
 'cv': StratifiedKFold(n_splits=5, random_state=149028763, shuffle=True),
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 2000,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l1',
 'random_state': 149028763,
 'refit': True,
 'scoring': 'neg_log_loss',
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0}

In [7]:
# Prediction task
X = []
for pair in tqdm(pairs):
    features = sum(map(feat_dict.get, pair))
    X.append(features)

X = sparse.vstack(X)

# Predict labels
predictions = clf.predict(X)

100%|██████████| 16296/16296 [00:03<00:00, 4921.43it/s]


In [8]:
# Examine prediction result
n_ppi = predictions.sum() # number of interactions reported
print('Predicted %i interactions (%.2f%% of candidate pairs)' % (n_ppi, (n_ppi/len(pairs) * 100)))

Predicted 10171 interactions (62.41% of candidate pairs)


In [9]:
# Obtain interacting pairs
interactions = np.array(pairs)[predictions.astype(bool)]

# Save interactions as DataFrame
f_out = os.path.join(dir_out, 'predicted_interactions.tsv')

df = pd.DataFrame(interactions, columns=['Pathogen_Uniprot_ID', 'Human_Uniprot_ID'])
df.to_csv(f_out, sep='\t', index=False)
df.head()

,Pathogen_Uniprot_ID,Human_Uniprot_ID
0,ENO_STRP2,FCAR_HUMAN
1,ENO_STRP2,FCG2B_HUMAN
2,ENO_STRP2,FCERG_HUMAN
3,ENO_STRP2,CLUS_HUMAN
4,ENO_STRP2,VTNC_HUMAN


<hr></hr>